# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [561]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [562]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [563]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [564]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [565]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [566]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [567]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [568]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 2

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [569]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]


In [570]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.fit_transform(y_train)
X_test = scaler.fit_transform(X_test)
y_test = scaler.fit_transform(y_test)

In [571]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [572]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [573]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()
number_units = window_size
dropout_fraction = 0.2

model.add(LSTM(units=number_units,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(dropout_fraction))

model.add(LSTM(units=number_units,return_sequences=True))
model.add(Dropout(dropout_fraction))

model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

model.add(Dense(1))

In [574]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [575]:
# Summarize the model
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_59 (LSTM)               (None, 2, 2)              32        
_________________________________________________________________
dropout_57 (Dropout)         (None, 2, 2)              0         
_________________________________________________________________
lstm_60 (LSTM)               (None, 2, 2)              40        
_________________________________________________________________
dropout_58 (Dropout)         (None, 2, 2)              0         
_________________________________________________________________
lstm_61 (LSTM)               (None, 2)                 40        
_________________________________________________________________
dropout_59 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)               

In [576]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended

#Set batch_size = 1
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=1,verbose=1)
model.evaluate(X_test, y_test)

Epoch 1/20
378/378 [==============================] - 1s 2ms/step - loss: 0.1272
Epoch 2/20
378/378 [==============================] - 1s 1ms/step - loss: 0.0690
Epoch 3/20
378/378 [==============================] - 1s 1ms/step - loss: 0.0649
Epoch 4/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0593
Epoch 5/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0446
Epoch 6/20
378/378 [==============================] - 1s 1ms/step - loss: 0.0352
Epoch 7/20
378/378 [==============================] - 1s 1ms/step - loss: 0.0203
Epoch 8/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0137
Epoch 9/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0177
Epoch 10/20
378/378 [==============================] - 1s 1ms/step - loss: 0.0164
Epoch 11/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0144
Epoch 12/20
378/378 [==============================] - 1s 2ms/step - loss: 0.0164
Epoch 13/20
378/378 [====

0.016348283737897873

In [577]:
#Set batch_size = 5
#model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=5,verbose=1)
#model.evaluate(X_test, y_test)

In [578]:
#try rmsprop optimizer
#model.compile(optimizer="rmsprop", loss="mean_squared_error")
#model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1,verbose=1)
#model.evaluate(X_test, y_test)

In [579]:
#try sgd optimizer
#model.compile(optimizer="sgd", loss="mean_squared_error")
#model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1,verbose=1)
#model.evaluate(X_test, y_test)

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [580]:
# Evaluate the model
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 831us/step - loss: 0.0163


0.016348283737897873

In [581]:
# Make some predictions
predicted = model.predict(X_test)

In [582]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [583]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
2019-02-18,3670.919922,4154.288574
2019-02-19,3912.570068,4164.377441
2019-02-20,3924.239990,4172.958496
2019-02-21,3974.050049,4222.062500
2019-02-22,3937.040039,4226.559082


In [584]:
# Plot the real vs predicted values as a line chart
stocks.hvplot.line( legend='top', height=500, width=620)

:NdOverlay   [Variable]
   :Curve   [index]   (value)